# Завдання

Проведіть класифікацію на основі датасету **UCI ML Breast Cancer Wisconsin**, використовуючи вказані у презентації моделі бустингу, беггінгу та стекінгу.

Змініть їх параметри наступним чином:

+ у **StackingClassifier** як базові алгоритми встановіть **LogisticRegression** та **DecisionTreeClassifier**, як фінальну модель – **SVC**;

+ у **BaggingClassifier** як базовий алгоритм встановіть **DecisionTreeClassifier**;

+ у **AdaBoostClassifier** як базовий алгоритм встановіть **LogisticRegression**;

+ у **XGBСlassifier** як базовий алгоритм (booster) встановіть **‘gbtree’**.

У зведеній таблиці порівняйте результати класифікацій, наведених у коді, із класифікаціями, що отримані вами.

# Розв'язок

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [2]:
data = load_breast_cancer()
data

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

## Підготовка до аналізу моделей

In [3]:
data, target = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=12)

## Стекінг

In [4]:
estimators = [('lgr', LogisticRegression(max_iter=10000)), ('dtr', DecisionTreeClassifier())]
model_stcl = StackingClassifier(estimators=estimators, final_estimator=SVC())
model_stcl.fit(X_train, y_train)

StackingClassifier(estimators=[('lgr', LogisticRegression(max_iter=10000)),
                               ('dtr', DecisionTreeClassifier())],
                   final_estimator=SVC())

In [16]:
score_stcl = round((model_stcl.score(X_test, y_test)), 6)
print(score_stcl)

0.935673


## Беггінг

In [8]:
model_bagcl = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=20, random_state=12)
model_bagcl.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=20,
                  random_state=12)

In [17]:
score_bagcl = round((model_bagcl.score(X_test, y_test)), 6)
print(score_bagcl)

0.906433


## Адаптивний бустинг

In [46]:
model_ada_bcl = AdaBoostClassifier(estimator = LogisticRegression(max_iter=1000), n_estimators=100, random_state=12)
model_ada_bcl.fit(X_train, y_train)

AdaBoostClassifier(estimator=LogisticRegression(max_iter=1000),
                   n_estimators=100, random_state=12)

In [47]:
score_ada_bcl = round((model_ada_bcl.score(X_test, y_test)), 6)
print(score_ada_bcl)

0.935673


## Градієнтний бустинг

In [61]:
model_xgdb_bcl = XGBClassifier(booster='gbtree', learning_rate=0.02, n_estimators=10)
model_xgdb_bcl.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [62]:
score_xgdb_bcl = round((model_xgdb_bcl.score(X_test, y_test)), 6)
print(score_xgdb_bcl)

0.912281


## Порівняльна таблиця

In [64]:
report = pd.DataFrame({
    'Ансамбль моделей': ['Стекінг', 'Баггінг', 'Адаптивний бустінг', 'Градієнтний бустинг'],
    'Алгоритми (лекція)': ['LogisticRegression, SGDClassifier, DecisionTreeClassifier', 'LogisticRegression', 'DecisionTreeClassifier', 'gblinear'],
    'Показник score (лекція)': [0.918129, 0.9356725, 0.918129, 0.865497],
    'Алгоритми (завдання)': ['LogisticRegression, DecisionTreeClassifier, SVC', 'DecisionTreeClassifier', 'LogisticRegression', 'gbtree'],
    'Показник score (завдання)': [score_stcl, score_bagcl, score_ada_bcl, score_xgdb_bcl],
    'Зміна показника score': [score_stcl-0.918129, score_bagcl-0.9356725, score_ada_bcl-0.9181287, score_xgdb_bcl-0.865497]
})
report

,Ансамбль моделей,Алгоритми (лекція),Показник score (лекція),Алгоритми (завдання),Показник score (завдання),Зміна показника score
0,Стекінг,"LogisticRegression, SGDClassifier, DecisionTre...",0.918129,"LogisticRegression, DecisionTreeClassifier, SVC",0.935673,0.017544
1,Баггінг,LogisticRegression,0.935673,DecisionTreeClassifier,0.906433,-0.029239
2,Адаптивний бустінг,DecisionTreeClassifier,0.918129,LogisticRegression,0.935673,0.017544
3,Градієнтний бустинг,gblinear,0.865497,gbtree,0.912281,0.046784


Таким чином, найкращий результат показали ансамблі моделей стекінгу та адаптивного бустингу, показники якості яких також вдалось дещо покращити, змінивши комбінації базових алгоритмів у порівнянні із кодом лекції.